In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
LOG_DIR = 'mnist_log'
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
# Standard functions as seen in normal mnist work    
def weight_variable(shape):
    # Because of reLU activations, we will initialize weights as positive
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def variable_summaries(var):
    """Attaches summaries to a Tensor"""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(var - mean))
        tf.summary.scalar('stdev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var)) # max along tensor
        tf.summary.scalar('min', tf.reduce_min(var)) # min in tensor
        tf.summary.histogram('histogram', var)

def nn_layer(input_tensor, input_dim, output_dim, layer_name, act=tf.nn.relu):
    """Reusable code for making a simple neural net layer.
    It does a matrix multiply, bias add, and then uses relu to nonlinearize.
    It also sets up name scoping so that the resultant graph is easy to read,
    and adds a number of summary ops.
    """
    # This ensures logical grouping of layers in the graph
    with tf.name_scope(layer_name):
        # This variable will hold the state of weights in layer
        with tf.name_scope('weights'):
            weights = weight_variable([input_dim, output_dim])
            variable_summaries(weights)
        with tf.name_scope('biases'):
            biases = bias_variable([output_dim])
            variable_summaries(biases)
        with tf.name_scope('Wx_plus_b'):
            preactivate = tf.matmul(input_tensor, weights) + biases
            tf.summary.histogram('pre_activations', preactivate)
        activations = act(preactivate, name='activation')
        tf.summary.histogram('activations', activations)
        return activations
    

In [5]:
def train():
    # importing data
    sess = tf.InteractiveSession()
    
    # Placeholders
    with tf.name_scope('input'):
        x = tf.placeholder(tf.float32, [None, 784], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, 10], name='y-input')
    
    with tf.name_scope('input_reshape'):
        image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
        tf.summary.image('input', image_shaped_input, 10)
    
    # create simple network
    
    # first fully connected layer
    hidden1 = nn_layer(x, 784, 500, 'layer1')
    
    with tf.name_scope('dropout'):
        keep_prob = tf.placeholder(tf.float32)
        tf.summary.scalar('dropout_keep_probability', keep_prob)
        dropped = tf.nn.dropout(hidden1, keep_prob)
    
    # second layer
    y = nn_layer(dropped, 500, 10, 'layer2', act=tf.identity)
    
    with tf.name_scope('cross_entropy'):
        # The raw formulation of cross-entropy,
        #
        # tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(tf.softmax(y)),
        #                               reduction_indices=[1]))
        #
        # can be numerically unstable.
        #
        # So here we use tf.nn.softmax_cross_entropy_with_logits on the
        # raw outputs of the nn_layer above, and then average across
        # the batch.
        diff = tf.nn.softmax_cross_entropy_with_logits(labels = y_, logits=y)
        with tf.name_scope('total'):
            cross_entropy = tf.reduce_mean(diff)
    tf.summary.scalar('cross_entropy', cross_entropy)
    
    with tf.name_scope('train'):
        train_step = tf.train.AdamOptimizer().minimize(cross_entropy)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)
    
    merged = tf.summary.merge_all() # merges all the summaries thus far
    
    train_writer = tf.summary.FileWriter(LOG_DIR + '/train', sess.graph)
    test_writer = tf.summary.FileWriter(LOG_DIR + '/test')
    tf.global_variables_initializer().run()
    
    
    def feed_dict(train):
        """Make a TensorFlow feed_dict, map the data onto the placeholders.
        Placeholders for this case are x, y_, keep_prob"""
        if train:
            xs, ys = mnist.train.next_batch(100, fake_data=False)
            k = 0.5
        else:
            xs, ys = mnist.test.images, mnist.test.labels
            k = 1.0
        
        return {x: xs, y_: ys, keep_prob: k}
    
    # Train the model, and write summaries
    # Every 10th step, measure test-set accuracy, write test summaries
    # All other steps, run train_step on training data & add summaries
    for i in range(2000):
        if i % 10 == 0:
            summary, acc = sess.run([merged, accuracy], feed_dict=feed_dict(False))
            test_writer.add_summary(summary, i)
            print('Accuracy at step %s : %s' % (i, acc))
        else:
            if i % 100 == 99:
                # record execution status
                run_options = tf.RunOptions(trace_level=tf.RunOptions.FULL_TRACE)
                run_metadata = tf.RunMetadata()
                summary, _ = sess.run([merged, train_step],
                                     feed_dict = feed_dict(True),
                                     options = run_options,
                                     run_metadata = run_metadata)
                train_writer.add_run_metadata(run_metadata, 'step%03d' % i)
                train_writer.add_summary(summary, i)
                print('Adding run metadata for', i)
            else:
                #record summary
                summary, _ = sess.run([merged, train_step], feed_dict=feed_dict(True))
                train_writer.add_summary(summary, i)
    train_writer.close()
    test_writer.close()
    

In [6]:
if tf.gfile.Exists(LOG_DIR):
    tf.gfile.DeleteRecursively(LOG_DIR)

tf.gfile.MakeDirs(LOG_DIR)
train()

Accuracy at step 0 : 0.1404
Accuracy at step 10 : 0.5984
Accuracy at step 20 : 0.7897
Accuracy at step 30 : 0.83
Accuracy at step 40 : 0.8459
Accuracy at step 50 : 0.8449
Accuracy at step 60 : 0.8768
Accuracy at step 70 : 0.8901
Accuracy at step 80 : 0.883
Accuracy at step 90 : 0.8929
Adding run metadata for 99
Accuracy at step 100 : 0.9033
Accuracy at step 110 : 0.9056
Accuracy at step 120 : 0.9082
Accuracy at step 130 : 0.9115
Accuracy at step 140 : 0.9171
Accuracy at step 150 : 0.9153
Accuracy at step 160 : 0.9221
Accuracy at step 170 : 0.9224
Accuracy at step 180 : 0.9216
Accuracy at step 190 : 0.9233
Adding run metadata for 199
Accuracy at step 200 : 0.9247
Accuracy at step 210 : 0.9309
Accuracy at step 220 : 0.929
Accuracy at step 230 : 0.9293
Accuracy at step 240 : 0.9335
Accuracy at step 250 : 0.9319
Accuracy at step 260 : 0.9322
Accuracy at step 270 : 0.9339
Accuracy at step 280 : 0.9347
Accuracy at step 290 : 0.9355
Adding run metadata for 299
Accuracy at step 300 : 0.9395
Ac